In [277]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [278]:
# load csv files
loading = pd.read_csv('/kaggle/input/trends-assessment-prediction/loading.csv')
train_scores = pd.read_csv('/kaggle/input/trends-assessment-prediction/train_scores.csv')
volumes_train = pd.read_csv('/kaggle/input/vol-data/volumes_df.csv')
volumes_test = pd.read_csv('/kaggle/input/vol-data/volumes_test_df.csv')

# replace nan with mean of columns
data_list = [loading, train_scores, volumes_train, volumes_test]

for data in data_list:
    for col in data.columns:
        data[col].fillna(data[col].mean(), inplace=True)
        
#drop columns from volumes with low variance
volume_list = [volumes_train]
list_50 = [str(i) for i in range(51)]

for x in volume_list:
    for j in list_50:
        if x.loc[:, j].var() / 100000 < 1.9:
            x.drop(columns=j, inplace=True)
            volumes_test.drop(columns=j, inplace=True)
            
# match

    
print(volumes_test.shape)
print(volumes_train.shape)

(5877, 2)
(5877, 2)


In [279]:
volumes_test.head()

,Id,1
0,10003,1418.0
1,10006,1905.0
2,10010,1410.0
3,10011,1872.0
4,10012,1132.0


In [280]:
volumes_train.head()

,Id,1
0,10001,1190.0
1,10002,1097.0
2,10004,1835.0
3,10005,1131.0
4,10007,1553.0


In [281]:
#split loading into train and test
loading_train = loading[loading.Id.isin(train_scores.Id)]
loading_for_pred = loading[~loading.Id.isin(train_scores.Id)]

In [282]:
# set up Xtrain X_for_pred and ytrain

X_train = loading_train.merge(volumes_train, on='Id').astype(np.float).set_index('Id')
X_for_pred = loading_for_pred.merge(volumes_test, on='Id').astype(np.float).set_index('Id')
y_train = train_scores.astype(np.float).set_index('Id')
X_train.head()

,IC_01,IC_07,IC_05,IC_16,IC_26,IC_06,IC_10,IC_09,IC_18,IC_04,...,IC_03,IC_21,IC_28,IC_11,IC_20,IC_30,IC_22,IC_29,IC_14,1
Id,,,,,,,,,,,,,,,,,,,,,
10001.0,0.006070,0.014466,0.004136,0.000658,-0.002742,0.005033,0.016720,0.003484,0.001797,0.029223,...,0.023711,0.009177,-0.013929,0.030696,0.010496,0.002892,-0.023235,0.022177,0.017192,1190.0
10002.0,0.009087,0.009291,0.007049,-0.002076,-0.002227,0.004605,0.012277,0.002946,0.004086,0.027333,...,0.022556,0.012004,-0.011814,0.022479,0.005739,0.002880,-0.016609,0.025543,0.014524,1097.0
10004.0,0.004675,0.000957,0.006154,-0.000429,-0.001222,0.011755,0.013010,0.000193,0.008075,0.027787,...,0.022266,0.005956,-0.010595,0.024078,-0.000319,0.005866,-0.015182,0.024476,0.014760,1835.0
10005.0,-0.000398,0.006878,0.009051,0.000369,0.000336,0.010679,0.010352,0.003637,0.004180,0.021058,...,0.017257,0.005454,-0.008591,0.019416,0.000786,0.002692,-0.019814,0.017105,0.013316,1131.0
10007.0,0.005192,0.010585,0.012160,-0.000920,-0.002255,0.011416,0.013838,0.001929,0.003051,0.031867,...,0.017332,0.016791,-0.009594,0.019757,0.003731,0.000733,-0.008462,0.026733,0.014358,1553.0


In [283]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold, GridSearchCV

features = ('age', 'domain1_var1', 'domain1_var2','domain2_var1','domain2_var2')

model = RandomForestRegressor(
    max_depth=15,
    min_samples_split=10,
    min_samples_leaf=3
)
cv = KFold(n_splits = 5, shuffle=True, random_state=42)
grid = {'n_estimators':[10,20,100, 200, 500]}
gs = GridSearchCV(model, grid, n_jobs=-1, cv=cv, verbose=10, scoring='neg_mean_absolute_error')

In [284]:
best_models = {}
for col in features:
    # if statement needed because it seems that the volume data helps all variables except domain1_var1 and domain2var2
    if col == 'domain1_var1' or col == 'domain2var2':
        gs.fit(X_train.iloc[:, :-1], y_train[col])
    else:
        gs.fit(X_train, y_train[col])
    best_models[col] = gs.best_estimator_
    print(gs.best_score_)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    3.6s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    7.0s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   40.0s
[Parallel(n_jobs=-1)]: Done  21 out of  25 | elapsed:  1.6min remaining:   18.3s
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:  2.3min finished


-8.02776357162907
Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    2.5s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    6.1s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   42.0s
[Parallel(n_jobs=-1)]: Done  21 out of  25 | elapsed:  1.7min remaining:   19.7s
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:  2.4min finished


-7.482515447209631
Fitting 5 folds for each of 5 candidates, totalling 25 fits


...

-8.401337134146655
Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    2.5s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    6.3s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   43.1s
[Parallel(n_jobs=-1)]: Done  21 out of  25 | elapsed:  1.8min remaining:   20.3s
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:  2.5min finished


-8.629669710186926
Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    6.5s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   44.4s
[Parallel(n_jobs=-1)]: Done  21 out of  25 | elapsed:  1.8min remaining:   20.8s
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:  2.6min finished


-9.21966048347145


In [285]:
predictions = pd.DataFrame(X_for_pred,columns=['Id'], dtype=str)

In [286]:
for col in features:
    predictions[col] = best_models[col].predict(X_for_pred)

In [287]:
predictions.head()

,Id,age,domain1_var1,domain1_var2,domain2_var1,domain2_var2
Id,,,,,,
10003.0,nan,47.645846,48.995909,60.237919,46.720017,52.785387
10006.0,nan,57.788706,50.389257,59.875033,52.779179,50.349617
10010.0,nan,47.192502,52.189609,59.888651,44.433476,51.826397
10011.0,nan,46.025680,49.658830,58.880925,46.244284,51.019457
10012.0,nan,55.417773,53.214187,57.418276,45.706065,53.539954


In [288]:
# stupid formatting work
predictions = predictions.drop(columns='Id')
predictions = predictions.reset_index('Id')
#predictions = predictions.rename(columns={'id': 'Id'})
predictions["Id"] = predictions["Id"].astype(int)
predictions.head()

,Id,age,domain1_var1,domain1_var2,domain2_var1,domain2_var2
0,10003,47.645846,48.995909,60.237919,46.720017,52.785387
1,10006,57.788706,50.389257,59.875033,52.779179,50.349617
2,10010,47.192502,52.189609,59.888651,44.433476,51.826397
3,10011,46.025680,49.658830,58.880925,46.244284,51.019457
4,10012,55.417773,53.214187,57.418276,45.706065,53.539954


In [289]:
def make_sub(predictions):
    features = ('age', 'domain1_var1', 'domain1_var2','domain2_var1','domain2_var2')
    _columns = (0,1,2,3,4)
    tests = predictions.rename(columns=dict(zip(features, _columns)))
    tests = tests.melt(id_vars='Id',value_vars=_columns,value_name='Predicted')
    tests['target'] = tests.variable.map(dict(zip(_columns, features)))
    tests['Id_'] = tests[['Id', 'target']].apply(lambda x: '_'.join((str(x[0]), str(x[1]))), axis=1)
  
    return tests.sort_values(by=['Id', 'variable'])\
              .drop(['Id', 'variable', 'target'],axis=1)\
              .rename(columns={'Id_':'Id'})\
              .reset_index(drop=True)\
              [['Id', 'Predicted']]

In [290]:
sub = make_sub(predictions)
sub.head()

,Id,Predicted
0,10003_age,47.645846
1,10003_domain1_var1,48.995909
2,10003_domain1_var2,60.237919
3,10003_domain2_var1,46.720017
4,10003_domain2_var2,52.785387


In [292]:
# do not forget to cast correct type
sub.to_csv('secondmodel_finetuned', index=False)